In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
def rmse(a, b):
    return np.sqrt(mean_squared_error(a, b))

In [7]:
train = pd.read_csv("../data/train/sales.csv").assign(date=lambda df: pd.to_datetime(df.date))
validation = pd.read_csv("../data/validation/sales.csv").assign(date=lambda df: pd.to_datetime(df.date))
test = pd.read_csv("../data/test/test.csv").assign(date=lambda df: pd.to_datetime(df.date))

In [39]:
items = pd.read_csv("../data/items.csv")
shops = pd.read_csv("../data/shops.csv")
item_categories = pd.read_csv("../data/item_categories.csv")

In [44]:
train.head().merge(items[["item_id", "item_category_id"]], on="item_id")

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
0,2013-02-01,0,59,22154,999.00,1.0,37
1,2013-03-01,0,25,2552,899.00,1.0,58
2,2013-05-01,0,25,2552,899.00,-1.0,58
3,2013-06-01,0,25,2554,1709.05,1.0,58
4,2013-01-15,0,25,2555,1099.00,1.0,56


In [8]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-02-01,0,59,22154,999.00,1.0
1,2013-03-01,0,25,2552,899.00,1.0
2,2013-05-01,0,25,2552,899.00,-1.0
3,2013-06-01,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2797945,2015-01-10,33,25,7640,4040.00,1.0
2797946,2015-05-10,33,25,7780,248.00,1.0
2797947,2015-04-10,33,25,7233,599.00,1.0
2797948,2015-03-10,33,25,7233,599.00,1.0


In [9]:
(train.query("item_cnt_day > 0")
    .assign(date=lambda df: pd.to_datetime(df.date))
    .assign(date_block_num=lambda df: df.eval("date.dt.year").astype(str) + "-" + df.eval("date.dt.month").astype(str))
)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-02-01,2013-2,59,22154,999.00,1.0
1,2013-03-01,2013-3,25,2552,899.00,1.0
3,2013-06-01,2013-6,25,2554,1709.05,1.0
4,2013-01-15,2013-1,25,2555,1099.00,1.0
5,2013-10-01,2013-10,25,2564,349.00,1.0
...,...,...,...,...,...,...
2797945,2015-01-10,2015-1,25,7640,4040.00,1.0
2797946,2015-05-10,2015-5,25,7780,248.00,1.0
2797947,2015-04-10,2015-4,25,7233,599.00,1.0
2797948,2015-03-10,2015-3,25,7233,599.00,1.0


In [10]:
train.date.dt.year.min()

2013

In [100]:
def aggregate_months(df):
    return (df.query("item_cnt_day > 0")
        .assign(
            date_month=lambda df: df.date.dt.month,
            date_year=lambda df: df.date.dt.year,
        )
        .merge(items[["item_id", "item_category_id"]], on="item_id")
        .assign(
            date_block_num=lambda df: (df.date_year - 2013) * 12 + df.date_month,
        )
        .assign(date_block_num=lambda df: df.date_block_num.astype(int))
        .groupby(["shop_id", "item_id", "date_block_num"])
        .agg({
            "item_cnt_day": sum, 
            "date_year": lambda r: r.iloc[0], 
            "date_month": lambda r: r.iloc[0], 
            "item_price": np.mean
        })
        .rename(columns={"item_cnt_day": "item_cnt_month"})
        .reset_index()
        .sort_values(["date_block_num", "shop_id", "item_id"])
        .assign(item_cnt_month=lambda df: df.item_cnt_month.clip(0, 20))
    )

In [101]:
train_df = aggregate_months(train)
X_train, y_train = train_df.drop(columns=["item_cnt_month"]), train_df["item_cnt_month"]

In [102]:
X_train

,shop_id,item_id,date_block_num,date_year,date_month,item_price
2,0,32,1,2013,1,221.000000
8,0,33,1,2013,1,347.000000
12,0,35,1,2013,1,247.000000
22,0,43,1,2013,1,221.000000
25,0,51,1,2013,1,128.500000
...,...,...,...,...,...,...
1698805,59,22087,32,2015,8,119.000000
1698836,59,22088,32,2015,8,115.666667
1698878,59,22092,32,2015,8,179.000000
1698881,59,22102,32,2015,8,1250.000000


In [103]:
val_df = aggregate_months(test)
X_val, y_val = val_df.drop(columns=["item_cnt_month"]), val_df["item_cnt_month"]

In [104]:
gb_reg = GradientBoostingRegressor(n_estimators=100)

In [105]:
gb_reg.fit(X_train, y_train)

GradientBoostingRegressor()

In [106]:
prediction = gb_reg.predict(X_val).clip(0, 20)

In [110]:
X_val

,shop_id,item_id,date_block_num,date_year,date_month,item_price
0,2,31,34,2015,10,399.0
1,2,67,34,2015,10,149.0
2,2,486,34,2015,10,300.0
3,2,787,34,2015,10,420.0
4,2,792,34,2015,10,979.0
...,...,...,...,...,...,...
34051,59,22088,34,2015,10,119.0
34052,59,22091,34,2015,10,179.0
34053,59,22100,34,2015,10,629.0
34054,59,22102,34,2015,10,1250.0


In [111]:
test_range = pd.read_csv("../data/test.csv", index_col=["shop_id", "item_id"])
target_df = test_range.join(val_df.set_index(["shop_id", "item_id"]).assign(prediction=prediction)).assign(
    date_block_num=(24 + 9), 
    item_cnt_month=lambda df: df.item_cnt_month.fillna(0),
    prediction=lambda df: df.prediction.fillna(0),
    date_year=2015, 
    date_month=9,
)

In [112]:
rmse(target_df.item_cnt_month, target_df.prediction)

0.734020740592809

In [98]:
target_df

ID  date_block_num  item_cnt_month  date_year  \
shop_id item_id                                                      
5       5037          0              33             0.0       2015   
        5320          1              33             0.0       2015   
        5233          2              33             0.0       2015   
        5232          3              33             0.0       2015   
        5268          4              33             0.0       2015   
...                 ...             ...             ...        ...   
45      18454    214195              33             2.0       2015   
        16188    214196              33             0.0       2015   
        15757    214197              33             0.0       2015   
        19648    214198              33             0.0       2015   
        969      214199              33             0.0       2015   

                 date_month  item_price  item_cnt_day  prediction  
shop_id item_id                                                    
5       5037              9         NaN           NaN    0.000000  
        5320              9         NaN           NaN    0.000000  
        5233              9         NaN           NaN    0.000000  
        5232              9         NaN           NaN    0.000000  
        5268              9         NaN           NaN    0.000000  
...                     ...         ...           ...         ...  
45      18454             9       149.0           2.0    1.996212  
        16188             9         NaN           NaN    0.000000  
        15757             9         NaN           NaN    0.000000  
        19648             9         NaN           NaN    0.000000  
        969               9         NaN           NaN    0.000000  

[214200 rows x 8 columns]